# Hybrid GA-SVR model

Here, the hybradized model of GA (Genetic Algorithm) and SVR (Support Vector Machine) is used to optimize the feature set and the hyper parameter of SVR model. This model is called as Hybradized model because we are using the SVR model in order to calculate the fitness value of each chromosomes inside Genetic Algorithm (Optimization Phase). And the same SVR model is used to predict the future values (training and testing phase).

Each chromosome is made up of number of features and svr parameters. The chromosomes structure can be defined as: 

                        |f1|f2|f3|f4|...|fn|p1|p2|p3|
                        
The features are represented as f1 to fn and the parameters are p1,p2 and p3. Intially, all the feature values are random values between 0 to 1. A threshold value is taken in order to select the features for model building phase.
For the SVR parameters, a range has been chosen.  

The total population (set of chromosomes) selected as 30 and GA operations applied for 100 rounds.

In [12]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.svm import SVR
import random
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from scipy import interpolate
import matplotlib.pyplot as plt

We have multivariate timeseries dataset where we have extracted features like year,month and day from the datetime feature in the datset.

In [13]:
#Read in data from CSV
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train['Datetime'] = pd.to_datetime(train.Datetime,format='%d-%m-%Y %H:%M')
test['Datetime'] = pd.to_datetime(test.Datetime,format='%d-%m-%Y %H:%M')

for i in (train,test):
    i['year']=i.Datetime.dt.year 
    i['month']=i.Datetime.dt.month 
    i['day']=i.Datetime.dt.day 

train= train.drop('Datetime',1)
test= test.drop('Datetime',1)

train = pd.DataFrame(train)
# test = pd.Dataframe(test)

train_norm = (train - train.min()) / (train.max() - train.min())
y_train = train_norm['PM10']
X_train  = train_norm.drop('PM10', axis = 1)

test_norm = (test - test.min()) / (test.max() - test.min())
y_test = test_norm['PM10']
X_test  = test_norm.drop('PM10', axis = 1)

  Here, we have taken the population size 30 where each chromosomes size is the sum of number of features available in the datset including extracted features and svr hyper parameter. Last three parameters are svr hyper parameters C (regularization parameter), epsoilon and gamma.

In [14]:
init_svr =[[-4.162666940178137,
  -4.306530277447223,
  -0.8520003036106027,
  -1.4770554389604107,
  -3.9499054323883342,
  -1.8494233120312376,
  1.2931671184911506,
  -0.9511554353798823,
  -0.8993505683629284,
  2.921506456539584,
  -3.0272222136642335,
  2.742959323201964,
  -4.61979371347661,
  4.55485717181358,
  3.506723830668273,
  1.0083698508060053,
  269.579647323562,
  0.01970846060929194,
  0.2124531276646993],
 [-2.066855255605786,
  -2.054262315358705,
  3.324005873767735,
  -2.7148483622890094,
  -0.6131915731518331,
  -4.799124659503184,
  3.0619164822428413,
  -0.7436655417491114,
  -3.1142408050819803,
  -4.210948723131189,
  -0.7065627782411337,
  0.32768858641444165,
  -4.667322438331184,
  -0.6767097677439171,
  3.56424797981221,
  -2.528567026357732,
  624.0716862897561,
  0.10131105497499944,
  0.76426297775973],
 [-4.503002524273919,
  -0.4434732268608137,
  -2.8714457918247827,
  -0.3345558428813913,
  -1.400375090305459,
  -3.8229464081737055,
  1.2790609562265098,
  -0.12130389001234754,
  -3.618037462684155,
  -4.7551202387671045,
  0.30803099819779245,
  -4.8065950437530836,
  -1.6965683755494476,
  1.1123606784872786,
  3.850928812163092,
  4.986444514918652,
  788.0617451091413,
  0.8149432795787557,
  0.4723202538923093],
 [-3.5934463202942215,
  -0.9668804746960529,
  3.68888996409186,
  -1.5816168203702432,
  -1.5741404396968295,
  3.437939416660411,
  1.9830849060499576,
  0.14421930289017837,
  -3.3095364311241293,
  -1.3225566539588893,
  1.6473456498009682,
  4.70090348281394,
  1.8083302667662702,
  0.5715436586557434,
  -1.9843770070769748,
  3.2316373229809443,
  648.7404764790232,
  0.6345036236389318,
  0.6622063263364236],
 [1.761146318358378,
  4.658318794805451,
  -3.7231077879999184,
  -0.10422695512207891,
  -4.463101650676818,
  -4.058960277440253,
  1.9415318898964031,
  -4.381883753589709,
  1.13680610262635,
  0.1496938724775827,
  -4.378461342935653,
  -0.6362751198139618,
  0.6578449638241093,
  -2.6945193022702663,
  -1.6143945276829061,
  4.398118830779211,
  388.49461349988655,
  0.771473611303104,
  0.20244268842409938],
 [-3.1931254368339044,
  -3.3096672907040405,
  -0.03128692341894279,
  0.485982018634898,
  -2.4981231408415816,
  -2.5025421821063567,
  4.264379259204663,
  3.822541839728963,
  3.2623812191390424,
  2.8506521984990822,
  -2.9387006580279773,
  -3.8781723132299337,
  -4.703842860622415,
  -4.691535988627864,
  0.5857636179747345,
  -0.26597900939240127,
  496.2470871575172,
  0.23671205393120806,
  0.42805056391380053],
 [-4.157603772337948,
  -3.5872902028302,
  0.07794833623741937,
  -4.182651895274293,
  -2.8688357875529213,
  2.5877169840218928,
  2.0062721509409966,
  -3.1003418001968885,
  -3.5875137181787284,
  -2.907067006531471,
  3.568248345838388,
  1.570313523396237,
  -0.7093582936151055,
  3.6994193020760573,
  4.480093494169594,
  2.0206776966389084,
  872.3764540773475,
  0.6899632547190172,
  0.28931712141328975],
 [-3.4170970973465598,
  2.5144351746717533,
  2.633146237638675,
  -2.427891167452697,
  -1.983940727303175,
  2.85401333353665,
  4.725711413047364,
  -4.808934815766389,
  -0.2806388373810389,
  -1.5379982603190978,
  -3.796540034608029,
  1.1084994716098002,
  -1.369472926509733,
  3.1302666407636544,
  -3.9714888127434698,
  3.8807674955692733,
  56.949900942897855,
  0.06205855764692171,
  0.42598077411930885],
 [-3.392925792642354,
  -1.8937287484135523,
  -4.860950799766317,
  -0.3897290703421632,
  -2.507820586453774,
  -4.384050949928104,
  -4.161681146261009,
  4.395454431166353,
  -0.7001183426061175,
  -2.4685376844110554,
  -3.5832689392794093,
  1.1933096402379606,
  4.567566642789332,
  2.073147945756962,
  1.9896904041931718,
  -2.456567926903909,
  124.84215575241606,
  0.8850133807504478,
  0.26515217536839153],
 [-0.9781943838675549,
  4.6812400498264335,
  -2.9603748191352475,
  0.42490453094604863,
  -3.893111463702544,
  0.6606758574716451,
  2.6188011394002775,
  -0.9547744184669407,
  -0.6405873788337813,
  0.28653493652857875,
  2.713819390406993,
  -3.1125045049162905,
  -1.1109762144520885,
  4.8412721076346035,
  3.596925382522599,
  -1.1197579527716628,
  891.570996298251,
  0.2120756246919953,
  0.3516201590924828],
 [0.9844639528667933,
  4.163277413018127,
  -2.916064411918382,
  -2.44933238975067,
  -0.6138378296935869,
  -2.1434188504324867,
  3.970588564892793,
  -2.6748915255452976,
  -0.7611293620107311,
  -4.732413284018962,
  -4.062821481427376,
  -3.724719118939502,
  -2.775187695938225,
  -2.551707474709577,
  3.7731626213440475,
  -4.812970312132753,
  996.0724338291552,
  0.16649723325068463,
  0.09795878453150667],
 [4.709988426617816,
  -1.9735693748840113,
  -2.780724307848951,
  -4.256019344136563,
  -4.055440301342514,
  -4.173134494836635,
  3.007417911002916,
  -3.281361870938987,
  3.3655218446283204,
  -1.7654127454818624,
  -1.0274550852624733,
  -3.903439469721215,
  1.033173417035571,
  1.8978187797981043,
  1.0307050108568072,
  4.108694985266979,
  432.9143924365568,
  0.8719412596650622,
  0.8619273276547462],
 [0.11883789993970417,
  4.138877622520006,
  1.9708731485181508,
  -1.094148922482466,
  -4.678015565670233,
  -4.403665523526449,
  1.3651297990298303,
  0.9864998463538708,
  1.0697883599826543,
  3.4994729274570595,
  3.0104347987674096,
  -1.7833241362797803,
  4.869674880482153,
  -3.479339132810048,
  0.8591512456504411,
  1.955197356235871,
  286.68183397956614,
  0.6709190964662958,
  0.5896237738345256],
 [-0.7318555310911501,
  2.2385564219787923,
  3.2035092427465734,
  3.6663013099011277,
  -1.8154950819355897,
  -0.8131553228816166,
  -1.3315883330540212,
  -0.2970973540162003,
  -1.7083242812203627,
  2.9324690391920836,
  -4.827685065023659,
  2.7929890412639775,
  -4.907200907687947,
  2.3757989304858,
  -3.301301652214874,
  0.8794105401536578,
  167.03622269551002,
  0.40793860350585964,
  0.3438935087857911],
 [4.671687234068715,
  -1.1310459489946059,
  1.468913020677677,
  3.7988118437180773,
  3.1408555734399624,
  -0.8166163624212235,
  1.048683464329045,
  -1.6209386676026072,
  2.626203854316347,
  3.250155869413888,
  4.653920630875605,
  -4.678679532528916,
  -2.7837978229879443,
  -3.406509604223802,
  -2.849884089974133,
  0.7934442700405722,
  798.9172963080191,
  0.5681861098079546,
  0.24518052322125244],
 [1.6581284215980627,
  4.435048489635832,
  4.254558553783731,
  3.446502011416804,
  1.4153326967126816,
  0.12343272893024704,
  4.636608763821249,
  -1.9880366927602933,
  2.6872586295181646,
  4.211652367523389,
  1.8592241015794073,
  3.0852416887345715,
  -3.838251137401645,
  -2.593629873594805,
  -0.362459949661579,
  -3.092070269364202,
  590.9521625143611,
  0.5174675204894184,
  0.11385381474571601],
 [3.983752671530512,
  1.5454672987911477,
  0.1058678119500911,
  0.9565801476698166,
  4.021070247472181,
  3.8063847443243954,
  -4.965536470734078,
  -0.6534956523846303,
  4.154221555646798,
  -4.438428336719347,
  4.811279571767612,
  -1.9135087029567908,
  -1.741269194728421,
  2.4268401322937647,
  2.6540255901503285,
  -0.8468088808628584,
  727.8990609523746,
  0.16575860561917574,
  0.36712202889315504],
 [-4.654044291944015,
  0.14915304715391464,
  3.405849321386805,
  0.2883251310037416,
  2.6558482412149065,
  -4.841261912328839,
  -2.5835203414406505,
  -0.9952478369701732,
  1.4924789811958021,
  1.5519749544517483,
  1.519740720085796,
  -0.26964684493689806,
  -4.272329893179782,
  0.0969068750652049,
  -3.02549509971503,
  -1.5535859213451344,
  456.7753533425995,
  0.48546030701696746,
  0.8332395892342807],
 [0.6241075403264666,
  4.078547536826321,
  4.838212145305297,
  3.985257667741095,
  -0.2365376839061799,
  -4.693879222739543,
  -0.30818564619771394,
  0.49531655209788195,
  -3.237693695759342,
  -4.516221373338631,
  -0.4744186170345941,
  -3.201576954929932,
  4.821720645386996,
  2.241468694637275,
  1.0384620700823746,
  4.379523569556131,
  390.5941604423283,
  0.6030465488277328,
  0.46692922184087704],
 [2.860218848107295,
  -2.377404786306384,
  -3.3881926836561815,
  -4.836902230190492,
  4.888780999448919,
  -2.1875948548299826,
  0.6691294160197812,
  -3.709243186171266,
  -3.9477476195027403,
  -1.0410696593454896,
  -1.2653094758052505,
  -1.940713329402728,
  -3.1703534949387646,
  -2.2068305746770367,
  -3.279163913049251,
  3.5575179124877945,
  802.3538647589311,
  0.5515222288916013,
  0.8693373928270075],
 [-2.329039845457105,
  -2.713870026894878,
  4.9290147875889865,
  -3.7177410346184003,
  -3.7586059453112686,
  2.266104113777306,
  -0.8759479398950001,
  3.789219963142841,
  4.941856949582736,
  -4.002246194108221,
  -2.4715482094676267,
  -2.807235836197174,
  4.500846951957353,
  -2.7896161174988454,
  -0.06267455783442255,
  0.44263009537274733,
  73.08316127568398,
  0.5112470079347428,
  0.8809834976047485],
 [0.8177208331619434,
  -1.6687805238524147,
  -2.1919269506199757,
  3.561937907651732,
  -0.4723224170069056,
  1.8053100763158163,
  2.925113981969501,
  4.385041704545232,
  1.1243179243788584,
  -4.612980712419144,
  3.2905684666210337,
  -1.7766233773398632,
  -3.55760722208035,
  -2.6105298394853946,
  -4.322120714352846,
  2.7813428685254387,
  785.9175401058818,
  0.032864902177935106,
  0.7063278554865814],
 [-4.102002482095498,
  4.742626931978988,
  1.1164781533559331,
  0.43687778074944994,
  1.8311814970292914,
  -4.40335194434658,
  0.05634258450851348,
  2.095559205677616,
  0.2801787609702693,
  -1.0616963517970568,
  -4.8208803779330065,
  -3.519477140005999,
  0.23875929106078964,
  3.046406606201465,
  3.267592533513298,
  2.1806090499546436,
  549.6167185699401,
  0.15987559581950442,
  0.4499519124317197],
 [-0.05021372890618636,
  -1.9746569403463896,
  -4.270598410024319,
  3.1705806248925317,
  -1.0925101746112342,
  -1.7664925078437466,
  -3.4618286721077163,
  -0.03857939576355385,
  -4.161086262097026,
  0.6908724851844115,
  -4.6989218890134055,
  2.0720584103715396,
  -1.708362491170703,
  -2.2916259743258816,
  -1.086606252623307,
  1.9654957161593467,
  828.9197039423407,
  0.2991070417972362,
  0.5843241215605479],
 [4.4834901472616,
  -2.9357872776141622,
  1.8736417257789304,
  3.66131100102967,
  0.9419459427701629,
  0.9352428576722644,
  -0.34458956974471544,
  1.792762627798239,
  3.9634636274247477,
  -0.36309858446888477,
  2.2250734923528928,
  1.60255560756955,
  0.13859171808772786,
  3.549288170070044,
  -1.5404456291818747,
  2.8614654107682105,
  195.47913718397476,
  0.310366716474482,
  0.3136746628292296],
 [3.60313168864268,
  -2.7799412140076516,
  0.8666057843875938,
  4.386660320819676,
  2.9454514835229775,
  -3.0932271149063215,
  -1.355074475335889,
  -2.422487222177052,
  4.9277116922812265,
  -4.892986054730347,
  -0.608645509310934,
  -3.5287271753144758,
  -4.680011350379946,
  -4.627964134725064,
  4.396184942128533,
  1.9713756430002807,
  44.937819362751235,
  0.560282575973838,
  0.5163917369362775],
 [-3.407685018836567,
  3.9428149807948305,
  -4.0617102855014355,
  0.9995241291638273,
  0.7471628982601022,
  -3.805629201833287,
  -2.6859850029665133,
  -0.42383171462419256,
  -1.6092269632379628,
  -2.640355861284341,
  4.825802686760836,
  0.019424947334391263,
  2.235842014657421,
  -3.0057145812477994,
  -4.08386561371445,
  -0.008145743888724688,
  456.107901505075,
  0.3663164080868232,
  0.1783127383198145],
 [2.3408552098001243,
  -2.1358413056338375,
  4.695997904355641,
  -4.608132472597283,
  1.8843690130300157,
  -1.181026380992276,
  0.16530002792410237,
  -1.1075529076407986,
  3.2646694300061423,
  0.012082248506853688,
  -4.958366669529064,
  0.7587636122567893,
  -3.5131483119369156,
  1.6544305529066747,
  1.2172270462179027,
  -2.11731908910716,
  41.29713874487386,
  0.4670832316765357,
  0.10925444270695252],
 [0.39429535740300725,
  2.0062808160304453,
  0.5456280562909814,
  4.899494394318488,
  -2.1628377049782754,
  2.358513877461668,
  -3.331701839689718,
  4.096588659753088,
  3.9022250458868886,
  4.838739144956669,
  -4.281216622217605,
  -4.417255668101532,
  -4.716653000170929,
  -3.6233717259294904,
  -0.6314512268361767,
  -2.3554970910851933,
  968.3157859665498,
  0.5763389058032297,
  0.339506058086308],
 [0.24970454425262112,
  -0.8506778428659185,
  2.202533224169194,
  -2.720767720704411,
  -3.3235626840863155,
  -4.315573937190662,
  -0.2703731235767348,
  -1.6456503529057844,
  3.5927356671913095,
  -4.160688599643837,
  3.3567628143158323,
  4.8751412908253915,
  0.2737924149384119,
  -4.825210022161041,
  -0.8734467898704636,
  -2.8239493550331884,
  387.82795700107096,
  0.5638380340044528,
  0.5381125670535079]]

In the Hybradized GA_SVR model, the model is splitted in two parts: one is parameter optimization phase and other one is model training and testing phase. 

In model training phase, the fitness value of each chromosomes is calculated. For calculating the fitness value (i.e Mean Squared Error) the selected features and the svr parameters are used to train the svr model using 10 fold cross-validation and MSE is calulated. Roulette Wheel Selection is applied to select the chromosomes for next step.

Next, Single point crossover is used to change the gene values within the chromosomes with some crossover probability. Further, Simple Random Mutation is used to add the uniqueness to the chromosomes.

In [15]:
class BGA():
    """
    Hybradized genetic algorithm.
    User Guide:
    >> test = GA(pop_shape=(10, 10), method=np.sum)
    >> solution, fitness = test.run()
    """
    def __init__(self, pop_shape, p_c=0.8, p_m=0.2, max_round = 100, early_stop_rounds=None, verbose = None, maximum=False):
        if early_stop_rounds != None:
            assert(max_round > early_stop_rounds)
        self.pop_shape = pop_shape
        self.pop = np.zeros(pop_shape)
        self.fitness = np.zeros(pop_shape[0])
        self.p_c = p_c
        self.p_m = p_m
        self.max_round = max_round
        self.early_stop_rounds = early_stop_rounds
        self.verbose = verbose
        self.maximum = maximum

    def evaluation(self, pop):
        fitness = []
        for i in range(len(pop)):
            individual = pop[i]
            cols = [index for index in range(16) if individual[index] <= 0.0]
            X_trainParsed = X_train.drop(X_train.columns[cols], axis=1)
            svr_rbf = SVR(kernel='rbf', gamma=individual[-1], C=individual[-3], epsilon=individual[-2]) 
            score = -1.0 * np.mean(cross_val_score(svr_rbf, X_trainParsed , y_train, scoring= 'neg_mean_squared_error', cv=10))
            fitness.append(score)
        return fitness
    
    def initialization(self):
        """
        Initalizing the population which shape is self.pop_shape(0-1 matrix).
        """
        self.pop = np.array(init_svr)
        self.fitness = self.evaluation(self.pop)

    def crossover(self, ind_0, ind_1):
        """
        Single point crossover.
        """
        assert(len(ind_0) == len(ind_1))
        point = np.random.randint(len(ind_0))
        new_0 = np.hstack((ind_0[:point], ind_1[point:]))
        new_1 = np.hstack((ind_1[:point], ind_0[point:]))
        assert(len(new_0) == len(ind_0))
        return new_0, new_1

    def mutation(self, indi):
        """
        Simple mutation.
        """
        point = np.random.randint(0,18)
        if point < 16:
            indi[point] = +5
        elif point == 16:
            indi[point] = 100
        elif point == 17:
            indi[point] = 0.01
        elif point == 18:
            indi[point] = 0.01
#        indi[point] = 1 - indi[point]
        return indi


    def rws(self, size, fitness):
        """
        Roulette Wheel Selection.
        Args:
            size: the size of individuals you want to select according to their fitness.
            fitness: the fitness of population you want to apply rws to.
        """
        if self.maximum:
            fitness_ = fitness
        else:
            fitness_ = fitness
        idx = np.random.choice(np.arange(len(fitness_)), size=size, replace=True,
               p=fitness_/np.sum(fitness_)) 
        return idx

    def run(self):
        """
        Run the genetic algorithm.
        """
        self.initialization()
        best_index = np.argsort(self.fitness)[0]
        global_best_fitness = self.fitness[best_index]
        global_best_ind = self.pop[best_index, :]
        count = 0

        for it in range(self.max_round):
            next_gene = []

            for n in range(int(self.pop_shape[0]/2)):
                i, j = self.rws(2, self.fitness) 
                # choosing 2 individuals with rws.
                indi_0, indi_1 = self.pop[i, :].copy(), self.pop[j, :].copy()
                if np.random.rand() < self.p_c:
                    indi_0, indi_1 = self.crossover(indi_0, indi_1)

                if np.random.rand() < self.p_m:
                    indi_0 = self.mutation(indi_0)
                    indi_1 = self.mutation(indi_1)

                next_gene.append(indi_0)
                next_gene.append(indi_1)

            self.pop = np.array(next_gene)
            self.fitness = self.evaluation(self.pop)

            if self.maximum:
                if np.max(self.fitness) > global_best_fitness:
                    best_index = np.argsort(self.fitness)[-1]
                    global_best_fitness = self.fitness[best_index]
                    global_best_ind = self.pop[best_index, :]
                    count = 0
                else:
                    count +=1
                worst_index = np.argsort(self.fitness)[-1]
                self.pop[worst_index, :] = global_best_ind
                self.fitness[worst_index] = global_best_fitness

            else:
                if np.min(self.fitness) < global_best_fitness:
                    best_index = np.argsort(self.fitness)[0]
                    global_best_fitness = self.fitness[best_index]
                    global_best_ind = self.pop[best_index, :]
                    count = 0
                else:
                    count +=1

                worst_index = np.argsort(self.fitness)[-1]
                self.pop[worst_index, :] = global_best_ind
                self.fitness[worst_index] = global_best_fitness
            
            print(it,'   ', global_best_fitness, global_best_ind)
            if self.verbose != None and 0 == (it % self.verbose):
                print('Gene {}:'.format(it))
                print('Global best fitness:', global_best_fitness)

            if self.early_stop_rounds != None and count > self.early_stop_rounds:
                print('Did not improved within {} rounds. Break.'.format(self.early_stop_rounds))
                break
            
        print('\n Solution: {} \n Fitness: {}'.format(global_best_ind, global_best_fitness))
        return global_best_ind, global_best_fitness
    

At each round, we are printing the glbal best chromosome and the fitness value of it. After selected number of rounds the final output gives the optimized feature set and optimized parameter values. These values can be further used to test the SVR model's performance on the testing dataset.

In [16]:
num_pop = 30
problem_dimentions = 19
test = BGA(pop_shape=(num_pop, problem_dimentions),  p_c=0.8, p_m=0.2, max_round = 100, early_stop_rounds=None, verbose = None, maximum=False)
best_solution, best_fitness = test.run()

0     0.011111795940062704 [-3.19312544e+00 -3.30966729e+00 -3.12869234e-02  4.85982019e-01
 -2.49812314e+00 -2.50254218e+00  4.26437926e+00  3.82254184e+00
  3.26238122e+00  2.85065220e+00 -2.93870066e+00 -3.87817231e+00
 -4.70384286e+00 -4.69153599e+00  5.85763618e-01 -2.65979009e-01
  4.96247087e+02  2.36712054e-01  4.28050564e-01]
1     0.011111795940062704 [-3.19312544e+00 -3.30966729e+00 -3.12869234e-02  4.85982019e-01
 -2.49812314e+00 -2.50254218e+00  4.26437926e+00  3.82254184e+00
  3.26238122e+00  2.85065220e+00 -2.93870066e+00 -3.87817231e+00
 -4.70384286e+00 -4.69153599e+00  5.85763618e-01 -2.65979009e-01
  4.96247087e+02  2.36712054e-01  4.28050564e-01]
2     0.010941684483181934 [ 2.34085521e+00 -2.13584131e+00  4.69599790e+00 -4.60813247e+00
  1.88436901e+00 -1.18102638e+00  1.65300028e-01 -1.10755291e+00
  3.26466943e+00  1.20822485e-02 -4.95836667e+00  7.58763612e-01
 -3.51314831e+00  1.65443055e+00  1.21722705e+00 -2.11731909e+00
  4.12971387e+01  1.00000000e-02  1.092

25     0.008421166200929208 [ 2.34085521e+00 -2.13584131e+00  3.40584932e+00 -3.89729070e-01
 -2.50782059e+00 -4.38405095e+00 -4.16168115e+00 -3.10034180e+00
 -3.58751372e+00 -2.90706701e+00  3.56824835e+00  1.57031352e+00
 -7.09358294e-01 -3.47933913e+00  8.59151246e-01 -2.11731909e+00
  4.12971387e+01  1.00000000e-02  1.09254443e-01]
26     0.008421166200929208 [ 2.34085521e+00 -2.13584131e+00  3.40584932e+00 -3.89729070e-01
 -2.50782059e+00 -4.38405095e+00 -4.16168115e+00 -3.10034180e+00
 -3.58751372e+00 -2.90706701e+00  3.56824835e+00  1.57031352e+00
 -7.09358294e-01 -3.47933913e+00  8.59151246e-01 -2.11731909e+00
  4.12971387e+01  1.00000000e-02  1.09254443e-01]
27     0.008421166200929208 [ 2.34085521e+00 -2.13584131e+00  3.40584932e+00 -3.89729070e-01
 -2.50782059e+00 -4.38405095e+00 -4.16168115e+00 -3.10034180e+00
 -3.58751372e+00 -2.90706701e+00  3.56824835e+00  1.57031352e+00
 -7.09358294e-01 -3.47933913e+00  8.59151246e-01 -2.11731909e+00
  4.12971387e+01  1.00000000e-02  1.

50     0.008421166200929208 [ 2.34085521e+00 -2.13584131e+00  3.40584932e+00 -3.89729070e-01
 -2.50782059e+00 -4.38405095e+00 -4.16168115e+00 -3.10034180e+00
 -3.58751372e+00 -2.90706701e+00  3.56824835e+00  1.57031352e+00
 -7.09358294e-01 -3.47933913e+00  8.59151246e-01 -2.11731909e+00
  4.12971387e+01  1.00000000e-02  1.09254443e-01]
51     0.008421166200929208 [ 2.34085521e+00 -2.13584131e+00  3.40584932e+00 -3.89729070e-01
 -2.50782059e+00 -4.38405095e+00 -4.16168115e+00 -3.10034180e+00
 -3.58751372e+00 -2.90706701e+00  3.56824835e+00  1.57031352e+00
 -7.09358294e-01 -3.47933913e+00  8.59151246e-01 -2.11731909e+00
  4.12971387e+01  1.00000000e-02  1.09254443e-01]
52     0.008421166200929208 [ 2.34085521e+00 -2.13584131e+00  3.40584932e+00 -3.89729070e-01
 -2.50782059e+00 -4.38405095e+00 -4.16168115e+00 -3.10034180e+00
 -3.58751372e+00 -2.90706701e+00  3.56824835e+00  1.57031352e+00
 -7.09358294e-01 -3.47933913e+00  8.59151246e-01 -2.11731909e+00
  4.12971387e+01  1.00000000e-02  1.

75     0.008421166200929208 [ 2.34085521e+00 -2.13584131e+00  3.40584932e+00 -3.89729070e-01
 -2.50782059e+00 -4.38405095e+00 -4.16168115e+00 -3.10034180e+00
 -3.58751372e+00 -2.90706701e+00  3.56824835e+00  1.57031352e+00
 -7.09358294e-01 -3.47933913e+00  8.59151246e-01 -2.11731909e+00
  4.12971387e+01  1.00000000e-02  1.09254443e-01]
76     0.008421166200929208 [ 2.34085521e+00 -2.13584131e+00  3.40584932e+00 -3.89729070e-01
 -2.50782059e+00 -4.38405095e+00 -4.16168115e+00 -3.10034180e+00
 -3.58751372e+00 -2.90706701e+00  3.56824835e+00  1.57031352e+00
 -7.09358294e-01 -3.47933913e+00  8.59151246e-01 -2.11731909e+00
  4.12971387e+01  1.00000000e-02  1.09254443e-01]
77     0.008421166200929208 [ 2.34085521e+00 -2.13584131e+00  3.40584932e+00 -3.89729070e-01
 -2.50782059e+00 -4.38405095e+00 -4.16168115e+00 -3.10034180e+00
 -3.58751372e+00 -2.90706701e+00  3.56824835e+00  1.57031352e+00
 -7.09358294e-01 -3.47933913e+00  8.59151246e-01 -2.11731909e+00
  4.12971387e+01  1.00000000e-02  1.

The best solution is now used to train the SVR model. After model training, the model is tested for its performace using the testing dataset. The performace can be tested using various parameter metrices like MSE, MAE, RMSE. 

In [ ]:
best= [ 2.34085521e+00, -2.13584131e+00,  3.40584932e+00, -3.89729070e-01,
 -2.50782059e+00, -4.38405095e+00 ,-4.16168115e+00 ,-3.10034180e+00,
 -3.58751372e+00 ,-2.90706701e+00  ,3.56824835e+00  ,1.57031352e+00,
 -7.09358294e-01, -3.47933913e+00  ,8.59151246e-01 ,-2.11731909e+00,
  4.12971387e+01 , 1.00000000e-02 , 1.09254443e-01]
cols=[]
for i in range(16):
    if(best[i] > 0.0):
        best[i] = 1
    else:
        best[i] = 0 
        cols.append(i)
print(best)
cols = [index for index in range(len(best)) if best[index] == 0]
X_trainParsed = X_train.drop(X_train.columns[cols], axis=1)
X_testParsed = X_test.drop(X_test.columns[cols], axis=1)
print(X_trainParsed.columns)

svr_rbf = SVR(kernel='rbf', gamma=best[-1], C=best[-3], epsilon=best[-2]) 
predictions = svr_rbf.fit(X_trainParsed, y_train).predict(X_testParsed)
y_test=np.array(y_test)

from sklearn.metrics import r2_score
print('r2:',r2_score(y_test, predictions)) 
y=y_test
mape=np.mean(np.abs((y - predictions) / y)) * 100
print('mape:',mape)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y, predictions)
print('MAE: %f' % mae)
from sklearn.metrics import mean_squared_error
from math import sqrt
mse = mean_squared_error(y, predictions)
print('MSE : %f' % mse)
rmse = sqrt(mse)
print('RMSE: %f' % rmse)
for i in range(len(predictions)):
    print(predictions[i])